In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import torch
pred:pd.DataFrame=pd.read_pickle('infer/reload_ckpt/seed-10043.pkl')

In [6]:
pred.columns

Index(['name', 'seq', 'taxo_label', 'domain_label', 'taxo', 'sentence_mask',
       'input_ids', 'attention_mask', 'Kingdom_predict', 'Phylum_predict',
       'Class_predict', 'Order_predict', 'predict_label'],
      dtype='object')

In [4]:
from hierataxo import  OrderManager
from hierataxo.dataset import ConcatProteinDataModule
import pickle as pkl
max_length=500
max_domain=15
train_bs=2
infer_bs=30

order_manager=OrderManager(pkl.load(open('taxo_data/hierarchy_order.pkl','rb'))['Riboviria'],
                        level_names=['Kingdom','Phylum','Class','Order'])

datamodule=ConcatProteinDataModule(order_manager,'taxo_data/proseq_taxo_1.pkl',
    max_length=max_length,max_domain=max_domain,train_bs=train_bs,infer_bs=infer_bs)
datamodule.setup('predict')
data=datamodule.dataset.data
# data[data['family'].apply(lambda x:len(x))>15].to_pickle('taxo_data/proseq_taxo_15more.pkl')

/home/taxo_playground/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/taxo_playground/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
data['family'].apply(lambda x:len(x)).max()

np.int64(28)

In [6]:
o=[]
for name,s in data.iterrows():
    for i,(seq,family) in enumerate(zip(s['seq'].upper().replace('-','').split('#'),s['family'])):
        o.append({
            'name':f'{name}-{i}-{family}',
            'seq':seq,
            'taxo':s['taxo'],
            'family':(family,)
        } )

data_single_domain=pd.DataFrame(o).set_index('name')
# data_single_domain.to_pickle('taxo_data/proseq_taxo_single_domain.pkl')
# data_single_domain[data_single_domain['seq'].apply(lambda x:len(x))>1000].to_pickle('taxo_data/proseq_taxo_single_domain_1000more.pkl')
data_single_domain['seq'].apply(lambda x:len(x)).describe()

count    22693.000000
mean       306.796369
std        225.866445
min          6.000000
25%        144.000000
50%        248.000000
75%        415.000000
max       2054.000000
Name: seq, dtype: float64

In [7]:
import matplotlib.pyplot as plt
from hierataxo.util import xkcd_color,hide_spline
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
    # "text.usetex": True,
    # "text.latex.preamble": r"\usepackage{amsmath}",
    'svg.fonttype':'none',
    'font.sans-serif':['Arial','Helvetica',
        'DejaVu Sans',
        'Bitstream Vera Sans',
        'Computer Modern Sans Serif',
        'Lucida Grande',
        'Verdana',
        'Geneva',
        'Lucid',
        'Avant Garde',
        'sans-serif'],
    "pdf.use14corefonts":False,
    'pdf.fonttype':42,
    'text.color':xkcd_color('dark grey'),
    'axes.labelweight':'heavy',
    'axes.titleweight':'extra bold'
        })

In [10]:
plt.close()
fig,ax=plt.subplots(1,1)
data_single_domain['seq'].apply(lambda x:len(x)).hist(ax=ax,color=xkcd_color('steel grey'),bins=40)
ax.set_xlabel('domain length')
ax.set_ylabel('count')
ax.grid(False)
hide_spline(ax=ax,hide='tr')
# plt.show()
fig.savefig('tmp_cm_ana/figs/domain_length.svg')
plt.close()

In [1]:
plt.close()
fig,ax=plt.subplots(1,1)
data['family'].apply(lambda x:len(x)).hist(ax=ax,color=xkcd_color('steel grey'),bins=10)
ax.set_xlabel('domain per segments')
ax.set_ylabel('count')
ax.grid(False)
hide_spline(ax=ax,hide='tr')
fig.savefig('tmp_cm_ana/figs/domain_count.svg')
fig.tight_layout()
plt.close()

NameError: name 'plt' is not defined

In [49]:
plt.rcParams['text.kerning_factor']

0

In [46]:
[i for i in plt.rcParams if 'text' in i]

['axes.formatter.use_mathtext',
 'legend.handletextpad',
 'mathtext.bf',
 'mathtext.bfit',
 'mathtext.cal',
 'mathtext.default',
 'mathtext.fallback',
 'mathtext.fontset',
 'mathtext.it',
 'mathtext.rm',
 'mathtext.sf',
 'mathtext.tt',
 'text.antialiased',
 'text.color',
 'text.hinting',
 'text.hinting_factor',
 'text.kerning_factor',
 'text.latex.preamble',
 'text.parse_math',
 'text.usetex']

In [51]:
plt.close()
fig, ax = plt.subplots()

ax.plot([1, 2, 3], label=r'$\sqrt{x^2}$')
ax.legend()

ax.set_xlabel(r'$\Delta_i^j$', fontsize=20)
ax.set_ylabel(r'$\Delta_{i+1}^j$', fontsize=20)
ax.set_title(r'$\Delta_i^j \hspace{0.4} \mathrm{versus} \hspace{0.4} '
             r'\Delta_{i+1}^j$', fontsize=20)

tex = r'$\mathcal{R}\prod_{i=\alpha_{i+1}}^\infty a_i\sin(2 \pi f x_i)$'
ax.text(1, 1.6, tex, fontsize=20, va='bottom')

fig.tight_layout()
# plt.show()
fig.savefig('math.svg')

In [54]:
from hierataxo.util import named_taxo_palette


for i,level_name in enumerate(order_manager.level_names):
    plt.close()
    dist=pred['taxo_label'].apply(lambda x:x[i]).value_counts().to_dict()
    dist['null']=0
    for i in list(dist.keys()):
        if 'Null' in i:
            dist['null']+=dist.pop(i)
    pie_l=[]
    for k,v in dist.items():
        pie_l.append([k,v,
                    named_taxo_palette.get(k,xkcd_color('grey')),
                    order_manager.graph_pos.get(k,[0,0])[1]])
    pie_l.sort(key=lambda x:x[1],reverse=True)
    # after_null=0
    # for i in range(len(pie_l)):
    #     if after_null==1:
    #         pie_l[i][0]=''
    #     if pie_l[i][0]=='null':
    #         after_null=1
    #         null_count=pie_l[i][1]
    for i in range(len(pie_l)):
        if pie_l[i][1]<60:
            pie_l[i][0]=''

    fig, ax = plt.subplots(1,1,figsize=(12,12))
    def autopct(x:float):
        c=x*len(pred)/100
        if c<60:
            return ''
        else:
            return f'{int(c)}'
        # if c<null_count:
        #     return ''
        # else:
        #     return f'{int(c)}'
    ax.pie(
        x=[i[1] for i in pie_l],
        labels=[i[0] for i in pie_l],
        colors=[i[2] for i in pie_l],
        autopct=autopct, #lambda x:f'{int(x*len(pie_l))}'
        # font_size=12
        textprops={'fontweight':'bold','fontsize':12}
        )
    ax.set_title(f'{level_name}-distribution')
    fig.savefig(f'tmp_cm_ana/figs/distribution-{level_name}.svg')
    # plt.show()
    plt.close()

In [27]:
pie_l

[['Orthornavirae', 6497, '#ffff43', (286.11, 324.86)],
 ['Pararnavirae', 135, '#c6afe9', (422.92, 280.41)],
 ['null', 28, 0.5843137254901961, 0]]

In [21]:
dist

{'Orthornavirae': 6497, 'Pararnavirae': 135, 'Null': 28}

In [17]:
order_manager.graph_pos

{'root': (355.52, 305.71),
 'Orthornavirae': (286.11, 324.86),
 'Duplornaviricota': (494.33, 344.02),
 'Chrymotiviricetes': (570.65, 325.07),
 'Ghabrivirales': (642.36, 331.53),
 'Resentoviricetes': (563.73, 363.17),
 'Reovirales': (633.14, 382.33),
 'Vidaverviricetes': (550.12, 399.43),
 'Mindivirales': (614.99, 430.67),
 'Kitrinoviricota': (423.75, 432.52),
 'Alsuviricetes': (504.78, 461.83),
 'Hepelivirales': (588.51, 474.99),
 'Martellivirales': (554.54, 513.87),
 'Tymovirales': (514.17, 546.07),
 'Flasuviricetes': (440.41, 504.33),
 'Amarillovirales': (468.71, 570.53),
 'Magsaviricetes': (403.58, 516.29),
 'Nodamuvirales': (419.6, 586.49),
 'Tolucaviricetes': (365.21, 521.49),
 'Tolivirales': (368.44, 593.42),
 'Lenarviricota': (276.19, 425.89),
 'Amabiliviricetes': (326.52, 519.76),
 'Wolframvirales': (316.86, 591.1),
 'Howeltoviricetes': (288.77, 511.14),
 'Cryppavirales': (266.52, 579.61),
 'Leviviricetes': (220.84, 474.59),
 'Norzivirales': (219.04, 559.32),
 'Timlovirales': (